In [21]:
## Created By Ahmed Al Hammadi under supoervision of Mr. Anshu Pandey during an AI Training Course
# Date: 22 - April - 2019

from sklearn.preprocessing import OneHotEncoder
import cv2
import numpy as np
import math
import os
import random

#################### Data Preprocessing ###########################################


# Simple Function to load an image using open CV library
def load_image(path):
    img= cv2.imread(path)
    return img
  
# a function to randomly crop an image with a specific width and height
def randomCrop(img, width, height):
    assert img.shape[0] >= height
    assert img.shape[1] >= width
    x = random.randint(0, img.shape[1] - width)
    y = random.randint(0, img.shape[0] - height)
    img = img[y:y+height, x:x+width]
    return img

# a function to locate the corresponding mask of the fake image and crop it with some extra margin 
def mask_crop(path):
    #Read and resize the image
    img = cv2.imread(path) 
    img = cv2.resize(img,(299,299))
    # Converts images from BGR to HSV
    hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    #Grayscale range for detection
    black_lower = np.array([0,0,0],np.uint8)
    black_upper = np.array([25,25,25],np.uint8)
    black = cv2.inRange(hsv, black_lower, black_upper)

    # Morphological Transform, Dilation
    kernal = np.ones((5, 5), "uint8")

    # The bitwise and of the frame and mask is done so
    # that only the red coloured objects are highlighted
    # and stored in res
    black = cv2.dilate(black, kernal)
    # detect the mask using fincCountours and extract the dimensions of the bounding bax to be using for cropping
    (_,contours, hierarchy)=cv2.findContours(black, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for pic, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        if(area > 50): #if detected area > 50 pixels proceed!
            x, y, w, h = cv2.boundingRect(contour)
            w2=math.ceil(0.01*w) # Take only 1% more margin
            h2=math.ceil(0.01*h)
            x2=x-math.ceil(0.05*x) # Recenter the bounding box
            y2=y-math.ceil(0.05*y)
        else:
            y2=None # to handle an error when there is no detection
    img2 = cv2.imread(path[:-9]+".jpg") # read the corresponding image to the mask
    img2 = cv2.resize(img2,(299,299))
    if y2:
        img = img2[y2:y + h + h2, x2:x + w + w2] # if bounding box exists, crop the fake portion
        img = cv2.resize(img,(299,299)) # Resize according to InceptionV3  requirements
        return img
    else:
        return None
#
folders=['fake','pristine'] # Define folders in directory
trainimg=[] # empty list for training data
trainlabel=[] # empty list for training labels
for folder in folders: # iterate in each folder 
    direc = "C:\\Users\\CUDA1\\Downloads\\training"+"\\"+folder
    images = os.listdir(direc)
    for path in images:
        if folder=='fake':
            if "mask" in path: # if mask is included in filename, apply mask crop
                img = mask_crop(direc+'/'+path)
                if img is not None:
                    trainimg.append(img)
                    trainlabel.append(folder)
        elif folder=='pristine':
            img = load_image(direc+'/'+path) # Otherwise, apply a similar random crop to the pristine image
            img = randomCrop(img,299,299)
            trainimg.append(img)
            trainlabel.append(folder)
            
trainimg = np.concatenate([trainimg])
trainlabel = np.array(trainlabel).reshape(-1,1)
trainlabel = OneHotEncoder().fit_transform(trainlabel).toarray()
trainlabel.shape


(1380, 2)

In [2]:
#################### Training a model to Learn-from-screach  ###########################################
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D

model = Sequential()
model.add(Conv2D(28, kernel_size=(3,3), input_shape=(299,299,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(28, kernel_size=(3,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation=tf.nn.relu))
model.add(Dropout(0.2))
model.add(Dense(2,activation=tf.nn.softmax))
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])
model.fit(x=trainimg,y=trainlabel,epochs=10,validation_split=0.2)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Using TensorFlow backend.


Instructions for updating:
Use tf.cast instead.
Train on 1116 samples, validate on 280 samples
Epoch 1/10
1116/1116 [==============================] - 10s 9ms/step - loss: 5.2439 - acc: 0.6738 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 2/10
1116/1116 [==============================] - 4s 4ms/step - loss: 4.9972 - acc: 0.6900 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 3/10
1116/1116 [==============================] - 4s 4ms/step - loss: 4.9972 - acc: 0.6900 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 4/10
1116/1116 [==============================] - 4s 4ms/step - loss: 4.9972 - acc: 0.6900 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 5/10
1116/1116 [==============================] - 4s 4ms/step - loss: 4.9972 - acc: 0.6900 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 6/10
1116/1116 [==============================] - 4s 4ms/step - loss: 4.9972 - acc: 0.6900 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 7/10
1116/1116 [==============================] - 4s 4ms/step - loss: 

In [10]:
#For Save the Model
model.save('keras_model_from_scratch.hdf5') 
 

C:\Users\CUDA1\Downloads


In [81]:
#################### Training a model using Transfer Learning ###########################################
import keras
from keras.models import load_model
from keras.models import Model
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input,decode_predictions
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.optimizers import SGD
opt = SGD(lr=0.001)

CLASSES = 2 #Fake or Pristine - Two Classes

# Load the model without the top layer which will be trained using our data
base_model = ResNet50(weights='imagenet', include_top=False) 

x = base_model.output

x = GlobalAveragePooling2D(name='avg_pool2')(x)

predictions = Dense(CLASSES, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)
   
#transfer learning
for layer in base_model.layers:
    layer.trainable = False

model.compile(opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(x=trainimg,y=trainlabel,epochs=35,validation_split=0.2)

Train on 924 samples, validate on 456 samples
Epoch 1/35
924/924 [==============================] - 49s 53ms/step - loss: 0.8271 - acc: 0.4670 - val_loss: 0.4947 - val_acc: 0.8037
Epoch 2/35
924/924 [==============================] - 14s 15ms/step - loss: 0.6944 - acc: 0.5860 - val_loss: 0.4195 - val_acc: 0.8969
Epoch 3/35
924/924 [==============================] - 14s 15ms/step - loss: 0.6171 - acc: 0.6580 - val_loss: 0.3868 - val_acc: 0.9189
Epoch 4/35
924/924 [==============================] - 14s 15ms/step - loss: 0.5637 - acc: 0.7181 - val_loss: 0.3596 - val_acc: 0.9397
Epoch 5/35
924/924 [==============================] - 14s 15ms/step - loss: 0.5219 - acc: 0.7478 - val_loss: 0.3437 - val_acc: 0.9474
Epoch 6/35
924/924 [==============================] - 14s 15ms/step - loss: 0.4854 - acc: 0.7868 - val_loss: 0.3344 - val_acc: 0.9463
Epoch 7/35
924/924 [==============================] - 14s 15ms/step - loss: 0.4514 - acc: 0.8187 - val_loss: 0.3249 - val_acc: 0.9496
Epoch 8/35
924/9

In [98]:
##################### Testing the model before deploying #############################

import cv2
import math
import numpy as np
from keras.preprocessing import image
from sklearn.feature_extraction import image as ims
def preprocess_input(x):
    x /= 255.
    x -= 0.5
    x *= 2.
    return x
#img_path = "C:\\Users\\CUDA1\\Downloads\\phoroa\\windsor.jpg"
#img_path = "C:\\Users\\CUDA1\\Downloads\\phoroa\\fake_putin.jpg"
#img_path = "C:\\Users\\CUDA1\\Downloads\\phoroa\\real_putin.jpg"
#img_path = "C:\\Users\\CUDA1\\Downloads\\phoroa\\Bush_fake.jpg"
#img_path = "C:\\Users\\CUDA1\\Downloads\\phoroa\\fake_tourist.jpg"

img =cv2.imread(img_path)
img = cv2.resize(img,(350,350))
nPatches=math.floor(img.size/(224*224))
patches = ims.extract_patches_2d(img, (224, 224))
threshold=0.1
predictions=[]
i=0
while (i<len(patches)):
    x = image.img_to_array(patches[i])
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    predictions.append(str(np.argmax(preds)))
    i=i+ (nPatches * 100)
s=0
j=0
m=0
while (s<len(predictions)):
    if (predictions[s])=='0':
        j=j+1
        print("real")
    if (predictions[s])=='1':
        print("fake")
        m=m+1
    s=s+1

real
fake
fake
fake
fake
fake
fake
real
real
fake
real
fake
real
fake
real
fake
real
fake
real
fake
real
fake
fake
fake


In [83]:
#################### Training a model using Transfer Learning ###########################################
import keras
from keras.models import load_model
from keras.models import Model
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input,decode_predictions
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.optimizers import SGD
opt = SGD(lr=0.001)

CLASSES = 2 #Fake or Pristine - Two Classes

# Load the model without the top layer which will be trained using our data
base_model = ResNet50(weights='imagenet', include_top=False) 

x = base_model.output

x = GlobalAveragePooling2D(name='avg_pool2')(x)

predictions = Dense(CLASSES, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)
   
#transfer learning
for layer in base_model.layers:
    layer.trainable = False

model.compile(opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(x=trainimg,y=trainlabel,epochs=10,validation_split=0.2)

Train on 1104 samples, validate on 276 samples
Epoch 1/10
1104/1104 [==============================] - 52s 47ms/step - loss: 0.7066 - acc: 0.5743 - val_loss: 0.5267 - val_acc: 0.7464
Epoch 2/10
1104/1104 [==============================] - 14s 13ms/step - loss: 0.5913 - acc: 0.6984 - val_loss: 0.4501 - val_acc: 0.8297
Epoch 3/10
1104/1104 [==============================] - 14s 13ms/step - loss: 0.5289 - acc: 0.7360 - val_loss: 0.4115 - val_acc: 0.8605
Epoch 4/10
1104/1104 [==============================] - 14s 13ms/step - loss: 0.4917 - acc: 0.7704 - val_loss: 0.3885 - val_acc: 0.8931
Epoch 5/10
1104/1104 [==============================] - 14s 13ms/step - loss: 0.4461 - acc: 0.7994 - val_loss: 0.3699 - val_acc: 0.9076
Epoch 6/10
1104/1104 [==============================] - 14s 13ms/step - loss: 0.4122 - acc: 0.8270 - val_loss: 0.3477 - val_acc: 0.9203
Epoch 7/10
1104/1104 [==============================] - 14s 13ms/step - loss: 0.3854 - acc: 0.8533 - val_loss: 0.3316 - val_acc: 0.9275
E

In [88]:
from keras.models import load_model
model = load_model("C:\\Users\\CUDA1\\Downloads\\keras_model4.hdf5")